# 4. Transfer learning na InceptionV3

In [ ]:
#sciagniecie wszystkich istotnych blbiotek
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import statistics
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#sciagniecie danych wykorzystancyh do transfer learningu modleu Incpetionv3
import pickle
pick_read = open('/content/gdrive/My Drive/X_train_color.pkl','rb')
X_train = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/X_val_color.pkl','rb')
X_val = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/X_test_color.pkl','rb')
X_test = pickle.load(pick_read)
pick_read.close()

pick_read = open('gdrive/My Drive/y_train_color.pkl','rb')
y_train = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/y_val_color.pkl','rb')
y_val = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/y_test_color.pkl','rb')
y_test = pickle.load(pick_read)
pick_read.close()

In [ ]:
#utworzenie modelu Inceptionv#
inception=InceptionV3(input_shape=(200,200,3),include_top=False, weights='imagenet')

#utworzenie warstw, ktorych wagi nie beda sie zmieniac w procesie trenowania sie modelu (wykluczajac warstwy 
#w pelni polaczone)
for layer in inception.layers:
    layer.trainable=False

#utworzenie czesci warstw w pelni polaczonych
layers= Flatten()(inception.output)
layers=Dense(1024,activation='relu')(layers)
layers=Dropout(0.2)(layers)
layers=Dense(50,activation='softmax')(layers)
inception=tf.keras.Model(inception.input,layers)
inception.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#zastosowanie metody early stopping
early_stop = EarlyStopping(monitor='val_loss',
                           min_delta=0, 
                           patience=10, 
                           verbose = 1, 
                           mode='min', 
                           restore_best_weights = True)

# wyuczenie modelu
history=inception.fit(
              X_train, 
              y_train, 
              batch_size = 50, 
              epochs = 100, 
              validation_data=(X_val, y_val), 
              callbacks=[early_stop])

#zapisanie wynikow modelu w formie pickle
pick_insert = open('gdrive/My Drive/model_inc.pkl','wb')
pickle.dump(inception, pick_insert)
pick_insert.close()

pick_insert = open('gdrive/My Drive/history_inc.pkl','wb')
pickle.dump(history, pick_insert)
pick_insert.close()

Epoch 1/100
348/348 [==============================] - 15s 29ms/step - loss: 5.0727 - accuracy: 0.1251 - val_loss: 2.7483 - val_accuracy: 0.2244
Epoch 2/100
348/348 [==============================] - 8s 22ms/step - loss: 2.8333 - accuracy: 0.2056 - val_loss: 2.5861 - val_accuracy: 0.2410
Epoch 3/100
348/348 [==============================] - 8s 22ms/step - loss: 2.6594 - accuracy: 0.2321 - val_loss: 2.4776 - val_accuracy: 0.2689
Epoch 4/100
348/348 [==============================] - 8s 22ms/step - loss: 2.5059 - accuracy: 0.2664 - val_loss: 2.3904 - val_accuracy: 0.2960
Epoch 5/100
348/348 [==============================] - 7s 21ms/step - loss: 2.3977 - accuracy: 0.2915 - val_loss: 2.4216 - val_accuracy: 0.3004
Epoch 6/100
348/348 [==============================] - 8s 22ms/step - loss: 2.2899 - accuracy: 0.3139 - val_loss: 2.4217 - val_accuracy: 0.2954
Epoch 7/100
348/348 [==============================] - 8s 22ms/step - loss: 2.1853 - accuracy: 0.3387 - val_loss: 2.3181 - val_accuracy